In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


In [27]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [28]:
path='imdb.csv'

In [29]:
try:
    load_df = pd.read_csv(path,sep=';')
    where=load_df['Id'].iloc[-1]+1
except:
  where=1


In [30]:
def keywords(id):
    word=''
    key_url='https://www.imdb.com/title/tt'+id+'/keywords/'
    response_keyword=requests.get(key_url,headers=headers)
    content_keyword=response_keyword.content
    soup_keyword=BeautifulSoup(content_keyword,'html.parser')
    keyword_elements=soup_keyword.find_all('a',class_='ipc-metadata-list-summary-item__t')
    for element in keyword_elements:
     word += ', ' + element.text
    #print('keyword')
    return word[2:len(word)]

In [31]:
def release_info(soup):
   date='NaN'
   x= soup.find_all('li',class_='ipc-metadata-list__item ipc-metadata-list-item--link')
   for r in x:
      if 'Release date' in r.text:
       date=r.text.replace('Release date','')
       break
   return date

In [52]:
df={'Id':[],'Link':[],'Title':[],'Genres':[],'Director':[],'Cast':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}

In [33]:
def country(soup):
    r_items = soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    inde=0
    for r in r_items:
  
      if r.text == 'None':
        country_name= r_items[inde-1].text
        break
      else:
        country_name='NaN'
      inde=inde+1
    return country_name

In [34]:
movie_genres = ["Action","Adventure","Comedy","Drama","Horror","Science Fiction (Sci-Fi)","Fantasy","Romance","Mystery","Crime","Thriller","Animation","Musical","War","Western","Biography","Historical",
    "Documentary","Family","Sports","Superhero","Musical Comedy","Film Noir","Romantic Comedy (Rom-Com)","Fantasy Adventure","Action Comedy","Psychological Thriller","Science Fantasy",
    "Epic","Post-Apocalyptic","History"]

In [35]:
def genere_fun(soup):
 gen_list=' '
 try:
  genres = soup.find_all('span', class_='ipc-chip__text')
  for g in  genres:
    if g.text in movie_genres:
        gen_list=g.text+' '+gen_list
  return gen_list 
 except:
  return 'NaN'
    

In [36]:
def runtime_(soup):
 runtime_test=soup.find_all('li',class_='ipc-metadata-list__item')
 color='NaN'
 runtime="NaN"
 for i in runtime_test:
    if 'Runtime' in i.text:
     runtime=i.text.replace('Runtime','')   
    if 'Color' in i.text:
      color=i.text #.replace('Color',' ')
 return runtime,color

In [37]:
def cast(soup):
    div_elements=soup.find_all('a',class_='sc-bfec09a1-1 fUguci')
    list=[element.text for element in div_elements]
    if len(list)>4:
        return list[0:6]
    else:
     return list


In [38]:
# function to test all other function

where =1517268
for num in range(where,where-10,-1):
  len_num=7-len(str(num))
  id_='0'*len_num+str(num)
  w_url='https://www.imdb.com/title/tt'+id_+'/'
  response=requests.get(w_url,headers=headers)
  content=response.content
  soup=BeautifulSoup(content,'html.parser')
  #print(w_url,keywords_(soup))
  print(w_url)
  print(actor(soup))

  #break

https://www.imdb.com/title/tt1517268/
['Margot Robbie', 'Ryan Gosling', 'Issa Rae', 'Kate McKinnon', 'Alexandra Shipp', 'Emma Mackey']
https://www.imdb.com/title/tt1517267/
['Stan Berkowitz', 'Paul Dini', 'Rich Fogel', 'Charles Hatfield', 'Dwayne McDuffie', 'Glen Murakami']
https://www.imdb.com/title/tt1517266/
[]
https://www.imdb.com/title/tt1517265/
[]
https://www.imdb.com/title/tt1517264/
[]
https://www.imdb.com/title/tt1517263/
['Markus Lanz', 'Alfred Biolek', 'Roger Cicero', 'Hans-Dietrich Genscher', 'Musical Ich war Noch niemals in New York', 'Jenny Jürgens']
https://www.imdb.com/title/tt1517262/
['Jung-hun Choi', 'Kwon Chul', 'Suk-yun Hong', 'Shoko Ikezu', 'Kamou Junko', 'Jeong-hwa Kim']
https://www.imdb.com/title/tt1517261/
[]
https://www.imdb.com/title/tt1517260/
['Saoirse Ronan', 'Max Irons', 'Jake Abel', 'Diane Kruger', 'Rachel Roberts', 'Shyaam Karra']
https://www.imdb.com/title/tt1517259/
[]


In [47]:
%%time
i=1
start_time=time.time()
for num in range(where,100000000):
                            len_num=7-len(str(num))
                            id_='0'*len_num+str(num)
                            w_url='https://www.imdb.com/title/tt'+id_+'/'
                            try:
                              response=requests.get(w_url,headers=headers)
                              content=response.content
                              soup=BeautifulSoup(content,'html.parser')
                              Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            except:
                              print(i,w_url,'>> Error')
                              continue
                            print(i,w_url,'<< processing')
                            Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            Director=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')[0].text
                            Date=release_info(soup)
                            
                            Runtime_,colr=runtime_(soup)
                            try:
                             Rating=(soup.find_all('span',class_='sc-bde20123-1 iZlgcd')[0]).text
                            except:
                              Rating="NaN"
                            country_=country(soup)
                            df['Country'].append(country_)
                            key_words=keywords(str(id_))
                            i=i+1
                            df['Id'].append(id_)
                            df['Link'].append(w_url)
                            df['Genres'].append(genere_fun(soup))
                            df['Title'].append(Title)
                            df['Director'].append(Director)
                            df['Date'].append(Date)
                            df['Runtime'].append(Runtime_)
                            df['Color'].append(colr)
                            df['Rating'].append(Rating)
                            df['Cast'].append(cast(soup))
                            df['keywords'].append(key_words)
                            if i==100:
                              try:
                               main_df=pd.DataFrame(df)
                               save_df=pd.concat([load_df, main_df], ignore_index=True)
                               save_df.to_csv(path,index=False,sep=';',header=True)
                               load_df = pd.read_csv(path,sep=';')
                               print('>>',load_df.shape)
                               df={'Id':[],'Link':[],'Title':[],'Genres':[],'Director':[],'Cast':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}
                               end_time=time.time()
                               print('>>Time', (end_time-start_time)/100)
                               start_time=time.time()
                              except:
                                main_df=pd.DataFrame(df)
                                save_df=main_df
                                save_df.to_csv(path,index=False,sep=';',header=True)
                                load_df = pd.read_csv(path,sep=';')
                                print('>>',load_df.shape)
                                df={'Id':[],'Link':[],'Title':[],'Genres':[],'Director':[],'Cast':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}
                                print('File not found Solved')

                              i=1
                          

1 https://www.imdb.com/title/tt1517268/ << processing
2 https://www.imdb.com/title/tt1517269/ << processing
3 https://www.imdb.com/title/tt1517270/ << processing
4 https://www.imdb.com/title/tt1517271/ << processing
5 https://www.imdb.com/title/tt1517272/ << processing
6 https://www.imdb.com/title/tt1517273/ << processing
7 https://www.imdb.com/title/tt1517274/ << processing
8 https://www.imdb.com/title/tt1517275/ << processing
9 https://www.imdb.com/title/tt1517276/ << processing
10 https://www.imdb.com/title/tt1517277/ << processing
11 https://www.imdb.com/title/tt1517278/ << processing
12 https://www.imdb.com/title/tt1517279/ << processing
13 https://www.imdb.com/title/tt1517280/ << processing
14 https://www.imdb.com/title/tt1517281/ << processing
15 https://www.imdb.com/title/tt1517282/ << processing
16 https://www.imdb.com/title/tt1517283/ << processing
17 https://www.imdb.com/title/tt1517284/ << processing
18 https://www.imdb.com/title/tt1517285/ << processing
19 https://www.imdb

IndexError: list index out of range

In [51]:
rf={}
for key in df:
    rf[key]=len(df[key])
print(rf)

{'Id': 41, 'Link': 41, 'Title': 41, 'Storyline': 0, 'Genres': 41, 'Director': 41, 'Cast': 41, 'Date': 41, 'Runtime': 41, 'Rating': 41, 'Color': 41, 'Country': 41, 'keywords': 41}
